In [28]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
from icecream import ic
import json
from datetime import datetime
from copy import deepcopy
from shapely import wkt
load_dotenv()

True

## load variables - gabions
- data folder
- cleaned_df - for only gabions
- json template
- 

In [29]:
cd mini_projects/regreen_cleaning/brics3_migration/

[Errno 2] No such file or directory: 'mini_projects/regreen_cleaning/brics3_migration/'
/Users/victor/Documents/projects/mini_projects/regreen_cleaning/brics3_migration


In [30]:
data_folder = os.path.join(os.getcwd(),'data')
cleaned_csv = os.path.join(data_folder, 'final_range_data.csv')
dff = pd.read_csv(cleaned_csv)

#lambda function to filter for gabions
#for each row, is it a list/str, then is 'gabions' in it, if so, return the df.
cleaned_df = dff[
    dff["land_use_selection-intervention"]
    .apply(lambda x: isinstance(x, (list, str)) and "gabions" in x)
]

#gabions datasets
gabions_df = pd.read_csv(os.path.join(data_folder, 'brics_csvs', 'outputs','cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_gabions.csv'))

TEMPLATE_FILE = os.path.join(data_folder,"halfmoon_json_template.json")
with open(TEMPLATE_FILE, "r") as f:
    template = json.load(f)



## check gabions count

In [31]:
x = dff["land_use_selection-intervention"]
count = 0
for idx, i in enumerate(list(x)):
    if 'gabions' in i:
        count += 1
        ic(i, count)

ic('gabions count',len(cleaned_df))

ic| i: 'halfmoons gabions water_points swales gulley_plugging'
    count: 1
ic| i: 'gabions', count: 2
ic| i: 'gabions', count: 3
ic| i: 'gabions', count: 4
ic| i: 'gabions', count: 5
ic| i: 'gabions', count: 6
ic| i: 'gabions', count: 7
ic| i: 'gabions', count: 8
ic| 'gabions count', len(cleaned_df): 8


('gabions count', 8)

## enumerators

In [32]:
#sorted(list(dff['basic_info-name'].unique()))
##empty name and key dictionary
names_dict = {}
for idx, row in dff.iterrows():
    #ic({row["KEY"], row["basic_info-name"]})
    names_dict[row["KEY"]] = row["basic_info-name"]
#ic(names_dict)
name_uname_dict = {
    "Zakariye haji ali": "abdullahi.hassan@mardo.org", 
    "Issack Idow Ibrahim": "abdullahi.hassan@mardo.org",
    "Abdullahi Hassan Adan": "abdullahi.hassan@mardo.org",
    "Abdikheir Mohamed": "mukhtar.yusuf23",
    "Abdikheir Mohamed Ali": "mukhtar.yusuf23",
    "Abdikheir Mohamed Dahir": "mukhtar.yusuf23",
    "Ali": "mukhtar.yusuf23",
    "Ali Adan": "mukhtar.yusuf23",
    "Ali Aden": "mukhtar.yusuf23",
    "Fathi": "mukhtar.yusuf23", 
    "Fathi Abdirashid": "mukhtar.yusuf23",
    "Ahmed Omar Hassan": "AhmedOmar",
    "Maria Abdirahman Ali": "AhmedOmar",
    "ABSHIR ABDULLAH ALI": "Ahil",
    "Ahmed Ibrahim": "JubaFoundation",
    "Ahmed Ibrahim Mohamed": "JubaFoundation",
    "Abdiaziz Adan Hassa": "guudow",
    "Abdiaziz Adan Hassan": "guudow",
    "Abdi Hassan Adan": "AbdiHassan",
    "Abdullahi": "sharif1234",
    "Abdirashid Sheikh Mohamed": "NRMO",
    "Abdullahi Sharif": "sharif1234",
    "Abdullahi Sharif LLG": "sharif1234",
    "Abdullahi Sharif Noor LLG": "sharif1234",
    "Abdijalil": "Csxaashi",
    "Abdijalil said": "Csxaashi",
    "Abdikarim Mohamed Ismail": "samandoulgou",
    "Abdikarin Mohamed Ismail": "samandoulgou",
    "Hassan Omar Khadhib": "Khadhib",
    "Hassan Omar Khadhibr": "Khadhib",
    "Jamal Ali": "JamalAli",
    "Jamal Ali Tagal": "JamalAli",
    "Mohamud Abdullahi Ali": "Mohamud",
    "Moulid Abdullahi Abdi": "Moulidwadani",
    "Muktar Mohamednoor Rage": "Mrage1997",
    "Osman Ebey Omar": "iibey",
    "Yusuf Mahi munye": "Mcnbdn617660",
    "abdullahi Sharif LLG": "sharif1234",
    "osman Ebey Omar": "iibey",
}

## match and replace names with usernames
for k, v in names_dict.items():
    for key, value in name_uname_dict.items():
        if v == key:
            names_dict[k] = value
#ic(names_dict)

##enumerator dictionary
enumerator = {
        "id": 3726,
        "first_name": "First",
        "last_name": "Last",
        "gender": "Male",
        "age_category": "18 - 35",
        "phone_number": "0e15704046",
        "email": "test_mail@gmail.com",
        "country": "test",
        "organization": "Test",
        "username": "test",
        "type": "ENUMERATOR"
    }

enums = []
#create enumerator jsons
for key, value in names_dict.items():
    e = deepcopy(enumerator)
    e['username'] = value
    e['key'] = key
    enums.append(e)

#ic(enums)

## farming entity

In [33]:
farming_entity_list = []
for idx, i in enumerate(cleaned_df.to_dict(orient='records'), start=1):
    farming_entity = {}
    #ic(i['basic_info-organisation_name'])
    farming_entity['id'] = idx
    farming_entity['first_name'] = i['basic_info-organisation_name']
    farming_entity['middle_name'] = i['basic_info-organisation_name']
    farming_entity['last_name'] = i['basic_info-organisation_name']
    farming_entity['organization'] = 'INSTITUTION'
    farming_entity['key'] = i['KEY']
    
    farming_entity_list.append(
        {
            "farmingEntity": farming_entity, 
            'key': i['KEY']
        }
    )
    #ic(farming_entity_list)
    

## plot-gabions_details
- read df to dictionary, for matching keys, extract df values
### plot-points
- parse the geoshape column from cleaned df
- split and extract the values from each i
- append to json keys

## crops
- get matching rows based on key(id)
- for these rows if crop has crops, get its values and 'other' if any
    - every matching rows values are appended to a list(of all crops for that plot)
- clean 'and' from the crop types



In [34]:
import uuid
import copy
import re

plot_list = []
point_id = 1

records = cleaned_df.to_dict(orient='records')

count = 0
#iterate over every i to capture plot details
for idx, i in enumerate(records):
    plot_id = idx + 1
    count += 1

    # ---- BUILD PLOT DETAILS ----
    template_cp = copy.deepcopy(template[0])
    plot_details = template_cp['plot']

    plot_details['id'] = plot_id
    plot_details['key'] = i['KEY']
    plot_details['name'] = uuid.uuid4().hex
    plot_details['calculated_size'] = i['area_ha']
    plot_details['photo_url'] = i['plot_details-photo']
    plot_details['farmingEntityId'] = farming_entity['id']
    plot_details['livestock_allowed'] = False if i['land_use_selection-land_use'] == 'agriculture' else True
    plot_details['conservation_area'] = False

    #get only child row for this key
    matching_child_rows = gabions_df[gabions_df['PARENT_KEY'] == i['KEY']]

    #holds every row's crops
    all_crops = []
    """----------------------------------CROP STATUS ----------------------------------"""
    #iterate over matching child is to build crop status
    for x in pd.DataFrame(matching_child_rows).to_dict(orient='records'):
        #ic(x['PARENT_KEY'], x["KEY"], x["managements_gabions-crops_in_field9"], x["managements_gabions-other_crops"])

        if x['managements_gabions-crops_present9'] == 'no' or pd.isna(x['managements_gabions-crops_present9']):
            plot_details['has_crops'] = False
        elif x['managements_gabions-crops_present9'] == 'yes':
            plot_details['has_crops'] = True


        #get a list of all crops in field
        if pd.isna(x['managements_gabions-crops_in_field9']):
            crop_list = []
        else:
            crop_list = str(x['managements_gabions-crops_in_field9']).split(" ")
            ic(crop_list, x['KEY'])


        #check that 'other' crops exist
        if 'other' in crop_list:
            #split by comma or 'and'
            other_crops_value = re.split(r',|\band\b', x['managements_gabions-other_crops'])
            #strip whitespace
            other_crops_value = [c.strip() for c in other_crops_value if c.strip()]

            #get rid of 'and' if exists
            if 'and' in other_crops_value:
                other_crops_value.remove('and')

            #only add if not empty
            if other_crops_value != []:
                crop_list.remove('other')
                crop_list.extend(other_crops_value)
                #append crop list for this row to all crops
            all_crops.extend(crop_list)
            ic(all_crops)
        
        elif 'other' not in crop_list:
            all_crops.extend(crop_list)

        plot_details['crops'] = list(set(all_crops))

        
    """--------------------------------GEOSHAPE PARSING ----------------------------------"""
    geoshape_str = i['plot_details-polygon']
    
    def parse_geoshape(geoshape_str):
        #parse a geoshape string into a list of point dictionaries
        points = []
        parts = geoshape_str.split(';')

        for part in parts:
            part = part.strip()
            if not part:
                continue

            lat, lon, alt, acc = map(float, part.split(' '))

            points.append({
                "longitude": lon,
                "latitude": lat,
                "altitude": alt,
                "accuracy": acc
            })

        return points

    parsed_points = parse_geoshape(geoshape_str)

    points_list = []
    for p in parsed_points:
        points_list.append({
            "id": point_id,
            "plotId": plot_id,
            "longitude": p["longitude"],
            "latitude": p["latitude"],
            "altitude": p["altitude"],
            "accuracy": p["accuracy"]
        })
        point_id += 1


    """--------------------------------ADMINISTRATIVE DETAILS ----------------------------------"""
    administrative = plot_details['subCounty']
    administrative['subcounty_name'] = i['geography-district_name']
    administrative['county']['county_name'] = i['geography-region_name']


    #update plot_details with number of points and administrative info
    plot_details['points'] = points_list
    plot_details['subCounty'] = administrative
    
    # del plot_details['points']
    # del plot_details['subCounty']
    general_plot = {
        "plot":plot_details, 
        'key':i['KEY']
    }
    #ic(i['KEY'])
    plot_list.append(
        general_plot
    )


In [35]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(plot_list, f, indent=4)

str(gabions_df['managements_gabions-crops_in_field9'][0]).split(' ')

['nan']

# project

In [36]:
projects_list = []

names =[]
for row in cleaned_df.to_dict(orient='records'):
    name = row['land_use_selection-project_name']
    

    if name == 'terra':
        name = name.upper()
        id = 131
    elif name == 'brcis3':
        name = 'BRCiS III'
        id = 93
    names.extend([name])
    projects_list.append({
        "project": {
            "id": id,
            "project_name": name
        },
        "key": row['KEY']
    })

ic(set(names))

ic| set(names): {'BRCiS III'}


{'BRCiS III'}

## gabions 
missing 
- herbacious cover
- who manages >> used who established
- erosion type
- is_intervention_effective
- vertical_spacing
- horizontal_spacing

In [37]:
gabions_no_children = []
gabions_econtrol = []

#dictionary to map odk values to RA app schema values
mapping_dict = {
    "STICKS_BRANCH":"STICKS_PROTECTION",
    "DEBRIS_REMOVAL":"DEBRIS_REMOVAL",
    "INVASIVE_SPECIES":"INVASIVE_CONTROL",
    "WEEDING":"WEEDING",
    "WATERING":"WATERING",
    "PLANTING_SEEDLING":"PLANTING_ANNUAL_PERENNIAL_CROPS"
}


for idx, c in enumerate(records):

    cureent_status_list = []
    all_grasses = []
    all_trees = []
    gabions_child_list = []


    """-----------------------------------gabions DETAILS-------------------------------------"""
    # collect row gabion details, will be used to create list of gabion detail+current status for each repeating record
    parent_base_gabions_details = {
        "id": idx + 1,
        "key":c["KEY"],
        "erosion_control_type": "Gabions",
        "established_date": datetime.strptime(c['gabions-gabions_details-date'], "%d/%m/%Y").strftime("%Y-%m"),
        "material_used": [
            "Rocks"
        ],
        "other_material_used": "",
        "who_established": ["MALE", "FEMALE"] if c["gabions-gabions_details-manages"] == 'both' else "MALE" if c["gabions-gabions_details-manages"]=="male" else [],
        "other_who_established": "YOUTH" if c["gabions-gabions_details-youth_manages"] == 'yes' else "",
        "total_interventions": c["gabions-gabions_details-total"],
        "length": c["gabions-gabions_details-length"],
        "width": c["gabions-gabions_details-width"],
        "depth": c["gabions-gabions_details-depth"],
        "vertical_spacing": 0,
        "horizontal_spacing": 0,
        "labor_payment": c["gabions-gabions_details-paid_unpaid_labor"],        
        # "has_grass": any(grass_present),
        # "total_number_grasses_planted": total_num_grasses_planted,
        # "kg_grass_seeds_planted": total_kg_grass_seeds,
        # "has_trees": any(trees_present),
        # "total_number_different_trees_planted": total_num_tree_species,
        # "total_number_trees_planted": total_num_trees_planted,
        # "total_number_trees_survived": total_num_trees_survived,
        # "management_practices":  [v for k, v in mapping_dict.items() if k in mngmts] + (["OTHER"] if any(k not in mapping_dict.keys() for k in mngmts) else []),
        # "other_managements": " ".join([c for c in mngmts if c not in mapping_dict.keys()]),
        "usages":["MIGRATED_NOT_KNOWN"],
        "other_usages": "",
        "rangelandEntryId": 2
        # "currentStatus": cureent_status_list,
        # "grassEstablishment": all_grasses,
        # "treeEstablishment":all_trees   
    }



    #if c['KEY'] == "uuid:1975a725-83a5-4bdd-8305-35ac65b2cce9":
    """-----------------------------------gabions CHILD DETAILS-------------------------------------"""

    #for every record, get the matching child rows
    matching_child_rows = gabions_df[gabions_df['PARENT_KEY'] == c['KEY']]


    total_num_grasses_planted = 0
    total_kg_grass_seeds = 0.0
    total_num_tree_species = 0
    total_num_trees_planted = 0
    total_num_trees_survived = 0
    grass_present = []
    trees_present = []
    mngmts = []
    
    """-------------------------non-child details records extraction-------------------------"""
    #for rows with no child rows, create a default current status, add base parent details
    if matching_child_rows.empty:
        ic('no child rows for', c['KEY'])
        centroid = wkt.loads(c["geometry"]).centroid
        currentStatus = {
            "id": idx + 1,
            "key": c["KEY"],
            "herbaceous_cover": "ABSENT",
            "visible_erosion": False,
            "erosion_type": [],
            "other_erosion_type": "",
            "is_intervention_effective": False,
            "longitude": centroid.x,
            "latitude": centroid.y,
            "altitude": 0,
            "accuracy": 0,
            "photo_url": "test.jpg",
            "comments": f"state {c['geography-state_name']} community {c['geography-community_name']}",
            "econtrolId": 1,
            "grassCurrentStatus": [],
            "treeCurrentStatus": []
        }
        gabions_details = {
            **parent_base_gabions_details,
            "has_grass": True if total_num_grasses_planted > 0 else False,
            "total_number_grasses_planted": total_num_grasses_planted,
            "kg_grass_seeds_planted": total_kg_grass_seeds,
            "has_trees": True if total_num_trees_planted > 0 else False,
            "total_number_different_trees_planted": total_num_tree_species,
            "total_number_trees_planted": total_num_trees_planted,
            "total_number_trees_survived": total_num_trees_survived,
            "management_practices":  [v for k, v in mapping_dict.items() if k in mngmts] + (["OTHER"] if any(k not in mapping_dict.keys() for k in mngmts) else []),
            "other_managements": " ".join([c for c in mngmts if c not in mapping_dict.keys()]),
            "currentStatus": currentStatus,
            "grassEstablishment": [],
            "treeEstablishment": []
        }

        gabions_no_children.append(gabions_details)
        #exit the child loop
        continue
    
    idxx =0
    for x in matching_child_rows.to_dict(orient='records'):

        total_num_grasses_planted = 0 if pd.isna(x["grass_gabions-num_grass_species"]) else x["grass_gabions-num_grass_species"]
        total_kg_grass_seeds = 0 if pd.isna(x["grass_gabions-kg_grass_seeds"]) else x["grass_gabions-kg_grass_seeds"]
        total_num_tree_species = 0 if pd.isna(x["tree_gabion-num_tree_species"]) else x["tree_gabion-num_tree_species"]
        total_num_trees_planted = 0 if pd.isna(x["tree_gabion-num_trees_planted"]) else x["tree_gabion-num_trees_planted"]
        total_num_trees_survived = 0 if pd.isna(x["tree_gabion-num_tree_survived"]) else x["tree_gabion-num_tree_survived"]
        grass_present.append(True if str(x["tree_gabion-planted_choice9"]).strip() in ["both", "grasses"] else False)
        trees_present.append(True if str(x["tree_gabion-planted_choice9"]).strip() in ["both", "trees"] else False)
        mngmts.extend(str(x["managements_gabions-management_practices"]).upper().split()) if not pd.isna(x["managements_gabions-management_practices"]) else 'NAN'

        
        if pd.notna(x['gps-Longitude']):
            child_coords = {
                "longitude": x["gps-Longitude"],
                "latitude" : x['gps-Latitude'],
                "altitude" : x["gps-Altitude"],
                "accuracy" : x["gps-Accuracy"],
                "photo_url": x["photo"],
                "comments" : f"agric_field_state {x["managements_gabions-agricultural_field_state"] if pd.notna(x["managements_gabions-agricultural_field_state"]) else "MIGRATED_NOT_KNOWN"}"
            }
                

        """--------------TREE ESTABLISHMENT-------------"""
        tree_status_list = []
        #no trees
        grass_status_list = []
        """---------------GRASS_ESTABLISHMENT------------"""
        #no grasses
       
        """--------------------CURRENT STATUS--------------------"""
        idxx += 1
        comms = f" state {c['geography-state_name'].replace("â€¦", "").replace(".", "").replace("..", "")} community {c['geography-community_name']}"
        currentStatus= {
            "id": idxx,
            "key":c["KEY"],            
            "herbaceous_cover": "ABSENT",
            "visible_erosion": False,
            "erosion_type": [
            ],
            "other_erosion_type": "NONE",
            "is_intervention_effective": True,
            "longitude": child_coords["longitude"] if child_coords else wkt.loads(c["geometry"]).centroid.x,
            "latitude": child_coords["latitude"]  if child_coords else wkt.loads(c["geometry"]).centroid.y,
            "altitude": child_coords["altitude"]  if child_coords else 0,
            "accuracy": child_coords["accuracy"]  if child_coords else 0,
            "photo_url": child_coords["photo_url"] if child_coords else 'test.jpg',
            "comments": child_coords["comments"] + comms if child_coords else comms,
            "econtrolId": 1,
            "grassCurrentStatus":grass_status_list,
            "treeCurrentStatus":tree_status_list
        }
        
        #add every status to the to serve as establishment        
        all_grasses.extend(grass_status_list)

        ## update establishment grasses and trees with econtrol_id key
        establishment_grasses = []
        for i in enumerate(all_grasses):
            data = dict(i[1])
            data['econtrol_id'] = data.pop('econtrol_status_id')
            establishment_grasses.append(data)
        

        establishment_trees = []
        for i in enumerate(all_trees):
            data = dict(i[1])
            data['econtrol_id'] = data.pop('econtrol_status_id')
            establishment_trees.append(data)
            

        gabions_details = {
            **parent_base_gabions_details,
            "has_grass": True if total_num_grasses_planted > 0 else False,
            "total_number_grasses_planted": total_num_grasses_planted,
            "kg_grass_seeds_planted": total_kg_grass_seeds,
            "has_trees": True if total_num_trees_planted > 0 else False,
            "total_number_different_trees_planted": total_num_tree_species,
            "total_number_trees_planted": total_num_trees_planted,
            "total_number_trees_survived": total_num_trees_survived,
            "management_practices":  [v for k, v in mapping_dict.items() if k in mngmts] + (["OTHER"] if any(k not in mapping_dict.keys() for k in mngmts) else []),
            "other_managements": " ".join([c for c in mngmts if c not in mapping_dict.keys()]),
            "currentStatus": currentStatus,
            "grassEstablishment": establishment_grasses,
            "treeEstablishment": establishment_trees   
        }

        gabions_child_list.append(gabions_details)
    
    if gabions_child_list:
        ic(c["KEY"], len(gabions_child_list))
        gabions_econtrol.append(
            {
                "econtrol": gabions_child_list,
                "key": c['KEY']
            }
        )



print(f"Total {len(cleaned_df)} - no child recs {len(gabions_no_children)} =  {len(cleaned_df)- len(gabions_no_children)} with kids" )
gabions_econtrol.extend([{"econtrol": [i], "key" :i["key"]} for i in gabions_no_children])

#add child list details to overal gabions_econtrol



Total 8 - no child recs 7 =  1 with kids


ic| c["KEY"]: 'uuid:7f209d1a-995b-4b0d-aed3-6d832c69b9a5'
    len(gabions_child_list): 6
ic| 'no child rows for': 'no child rows for'
    c['KEY']: 'uuid:c057a054-841a-47b7-9a73-323edc2aa97f'
ic| 'no child rows for': 'no child rows for'
    c['KEY']: 'uuid:462595d8-2d63-4d31-a8d2-d9558f2cf792'
ic| 'no child rows for': 'no child rows for'
    c['KEY']: 'uuid:d04f0fa4-3d5e-4a54-884c-206853b0db20'
ic| 'no child rows for': 'no child rows for'
    c['KEY']: 'uuid:ebb84261-cb61-4373-a49b-5143ebc771a1'
ic| 'no child rows for': 'no child rows for'
    c['KEY']: 'uuid:c1703221-8e2a-4e5b-9c4b-6ea19acd86de'
ic| 'no child rows for': 'no child rows for'
    c['KEY']: 'uuid:7f7994f6-e4e6-4136-b881-0bc16590284c'
ic| 'no child rows for': 'no child rows for'
    c['KEY']: 'uuid:1d046361-3a8b-433d-99d7-b6a103beab88'


If a parent has 4 repeats, reproduce the parent 4 time s 

In [38]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(gabions_econtrol, f, indent=4)

## merge components into one json

In [39]:
# component lists
lists = [farming_entity_list, gabions_econtrol, plot_list, projects_list]

# The output dictionary
merged = {}

for lst in lists:
    ic(len(lst))
    for item in lst:
        k = item['key']

        #check if already updated
        if k not in merged:
            merged[k] = {}
            
        #merge the item into the existing dictionary
        merged[k].update(item)

final = list(merged.values())
for i in final:
    #match enumerators
    for l in enums:
        if i['key'] == l['key']:
            i['enumerator'] = l


    i["boma"] = []
    i["cgrazing"] = []
    i["microcatchment"]= []
    i["seedbank"] = []
    i["seeding"] = []
    i["waterpoint"] = []
    i["completed"] = True
    i["iremoval"] = []
    i["rangeModuleId"] = 1
    i["currentStep"]= "DONE"
    i["moduleId"] = 6
    i["is_revisit"] = False
    #get the date_collected
    for j in records:
        if i["key"] == j["KEY"]:
            i["date_collected"] = datetime.strptime(j['basic_info-date'], "%d/%m/%Y").strftime("%Y-%m-%d")
    #match project id
    for x in projects_list:
        if i['key'] == x['key']:
            i['projectId'] = x['project']['id']
    #match plot id
    for k in plot_list:
        if i['key'] == k['key']:
            i['plotId'] = k['plot']['id']
    #match farming_entity_id
    for h in farming_entity_list:
        if i['key'] == h['key']:
            i['farmingEntityId'] = h['farmingEntity']['id']
    #match enumerator id
    for e in enums:
        if i['key'] == e['key']:
            i['enumeratorId'] = e['id']


ic(len(final))


ic| len(lst): 8
ic| len(lst): 8
ic| len(lst): 8
ic| len(lst): 8
ic| len(final): 8


8

In [40]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(final, f, indent=4)

## clean up keys

In [41]:
#clean up keys
def remove_keys(obj, key="key"):
    if isinstance(obj, dict):
        obj.pop(key, None)
        for v in obj.values():
            remove_keys(v, key)
    elif isinstance(obj, list):
        for v in obj:
            remove_keys(v, key)
    return obj

finaal = [x for x in final if x['key'] == 'uuid:b4a2d545-66fe-4d23-825b-41025149b460']
finaal = [remove_keys(i) for i in final]

#add id
for idx, c in enumerate(finaal):
    c['id'] = idx + 1



In [42]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(finaal, f, indent=4)

In [43]:
for i in enumerate(finaal):
    if len(i[1].keys()) != 23:
        ic("ERROR")
    else:
        ic("✅")

ic| '✅'
ic| '✅'
ic| '✅'
ic| '✅'
ic| '✅'
ic| '✅'
ic| '✅'
ic| '✅'
